# Use SimSum Classification to Link FEBRL People Data

<a href="https://colab.research.google.com/github/rachhouse/intro-to-data-linking/blob/main/tutorial_notebooks/01_Link_FEBRL_Data_with_SimSum_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"></a>

In this tutorial, we'll link synthesized people datasets generated by the [Freely Extensible Biomedical Record Linkage (FEBRL)](https://sourceforge.net/projects/febrl/) project. The FEBRL-generated datasets represent cleaned datasets, so in this notebook, we will step through:
* data augmentation,
* blocking,
* comparing, and
* classification using the SimSum methodology.

## Google Colab Setup

In [ ]:
# Check if we're running locally, or in Google Colab.
try:
    import google.colab
    COLAB = True
except ModuleNotFoundError:
    COLAB = False
    
# If we're running in Colab, download the tutorial functions file 
# to the Colab session local directory, and install required libraries.
if COLAB:
    import requests
    
    tutorial_functions_url = "https://raw.githubusercontent.com/rachhouse/intro-to-data-linking/main/tutorial_notebooks/linking_tutorial_functions.py"
    r = requests.get(tutorial_functions_url)
    
    with open("linking_tutorial_functions.py", "w") as fh:
        fh.write(r.text)
    
    !pip install -q recordlinkage jellyfish altair

## Imports

In [ ]:
import itertools
import re

from typing import Dict, Tuple, Optional

import altair as alt
import jellyfish
import numpy as np
import pandas as pd
import recordlinkage as rl

# We have a couple helper functions from this file that we'll use for evaluation.
import linking_tutorial_functions as tutorial

## Define Filepaths

First, let's set up access to a few data resources that we'll need for the tutorial.

In [ ]:
TRAINING_DATASET_A, TRAINING_DATASET_B, TRAINING_LABELS = tutorial.get_training_data_paths(COLAB)

## Load (Cleaned) Training Datasets

We'll load our training datasets into pandas DataFrames. We want to be able to take advantage of pandas indexing as we link our data (plus, the `recordlinkage` package that we'll be using later needs input DataFrames to be indexed by record id), so we'll set an index on each training DataFrame.

As mentioned above, we can consider the cleaning step of linking to be already done - the data generated by FEBRL is in a consistent format, and equivalent attributes have been encoded in the same manner for the two synthesized people datasets.

In [ ]:
df_A = pd.read_csv(TRAINING_DATASET_A)
df_A = df_A.set_index("person_id_A")
df_A.head()

In [ ]:
df_B = pd.read_csv(TRAINING_DATASET_B)
df_B = df_B.set_index("person_id_B")
df_B.head()

## Load Training Ground Truth Labels

One of the advantages of synthesized data, especially for tutorials and learning, is that we have ground truth labels for data. (This is rarely the case when you encounter linking problems in the wild). We'll load our known true links into a pandas DataFrame below.

In [ ]:
df_ground_truth = pd.read_csv(TRAINING_LABELS)
df_ground_truth = df_ground_truth.set_index(["person_id_A", "person_id_B"])
df_ground_truth["ground_truth"] = df_ground_truth["ground_truth"].apply(lambda x: True if x == 1 else False)
df_ground_truth.head()

## Data Augmentation

Let's take a look at our data, and consider what we have currently available for blocking and comparing.

In [ ]:
df_A.head(n=2)

It would probably make sense to block on people's first and last name, but, as we've noted, the realities of data entry typos, nicknames, aliases, OCR mishaps, and speech-to-text blips mean that using an exact blocker isn't going to work well. These fields are prime candidates for phonetic encoding!

We'll use the python [jellyfish library](https://pypi.org/project/jellyfish/) to encode our `first_name` and `surname` fields via two phonetic encoding algorithms, [**Soundex**](https://en.wikipedia.org/wiki/Soundex) and [**NYSIIS**](https://en.wikipedia.org/wiki/New_York_State_Identification_and_Intelligence_System).

We could also use a truncated exact blocking approach with the `soc_sec_id` field. For this, we'll create a new attribute containing the last three digits of the SSid.

And lastly, we'll cast the `date_of_birth` field to a pandas Timestamp field so that we can compare it more easily down the road.

In [ ]:
def dob_to_date(dob: str) -> Optional[pd.Timestamp]:
    """ Transform string date in YYYYMMDD format to a pd.Timestamp.
        Return None if transformation is not successful.
    """
    date_pattern = r"(\d{4})(\d{2})(\d{2})"
    dob_timestamp = None
    
    try:
        m = re.match(date_pattern, dob.strip())
        if m:
            dob_timestamp = pd.Timestamp(int(m.group(1)), int(m.group(2)), int(m.group(3)))
    except:
        pass

    return dob_timestamp

In [ ]:
%%time

for df in [df_A, df_B]:
    
    # Update NaNs to empty strings or jellyfish will choke.
    df["surname"] = df["surname"].fillna("")
    df["first_name"] = df["first_name"].fillna("")

    # Soundex phonetic encodings.
    df["soundex_surname"] = df["surname"].apply(lambda x: jellyfish.soundex(x))
    df["soundex_firstname"] = df["first_name"].apply(lambda x: jellyfish.soundex(x))
    
    # NYSIIS phonetic encodings.    
    df["nysiis_surname"] = df["surname"].apply(lambda x: jellyfish.nysiis(x))
    df["nysiis_firstname"] = df["first_name"].apply(lambda x: jellyfish.nysiis(x))
    
    # Last 3 of SSID.
    df["ssid_last3"] = df["soc_sec_id"].apply(lambda x: str(x)[-3:].zfill(3) if x else None)
    df["soc_sec_id"] = df["soc_sec_id"].astype(str)
    
    # DOB to date object.
    df["dob"] = df["date_of_birth"].apply(lambda x: dob_to_date(x))

Let's take a look at a sample of our new columns:

In [ ]:
df_A.head(n=2)

## Blocking

Now that we've augmented our datasets, let's try some blocking! We'll use the python [`recordlinkage` library](https://github.com/J535D165/recordlinkage) for blocking. 

First, let's see how many candidate record pairs we would generate with a full blocker - meaning if we compared every record in dataset A to every record in dataset B. This produces the [Cartesian product](https://en.wikipedia.org/wiki/Cartesian_product) of the two datasets.

In [ ]:
indexer = rl.Index()
indexer.add(rl.index.Full())

full_blocker_pairs = indexer.index(df_A, df_B)
max_candidate_record_pairs = full_blocker_pairs.shape[0]

print("\ndataset A size * dataset B size = maximum candidate record pairs")
print(f"{df_A.shape[0]:,} * {df_B.shape[0]:,} = {df_A.shape[0]*df_B.shape[0]:,}")

print(f"\n{max_candidate_record_pairs:,} total pairs.")

`indexer.index` returns a pandas MultiIndex of the candidate record pairs: 

In [ ]:
full_blocker_pairs

Even for very small datasets, like our training data, we're looking a huge amount of candidate record pairs to compare, unless we employ more selective blocking.

Recall that successful and efficient blocking minimizes:
* the quantity of generated candidate record pairs
* missed true links

So, first let's define a method which measures the percentage of true links captured by blocking, as well as the search space reduction.

In [ ]:
def evaluate_blocking(
    candidate_pairs: pd.MultiIndex,
    df_left: pd.DataFrame,
    df_right: pd.DataFrame,
    df_true_links: pd.DataFrame
) -> Tuple[float, float]:
    """ Function to calculate blocking search space reduction and retained true links.
        Reports and returns search space reduction percentage and retained true links percentage.
    """
    
    # Calculate search space reduction.
    search_space_reduction = round(rl.reduction_ratio(candidate_pairs.shape[0], df_left, df_right), 3)
    
    # Calculate retained true links percentage.
    total_true_links = df_true_links.shape[0]
    true_links_after_blocking = pd.merge(
        df_true_links,
        candidate_pairs.to_frame(),
        left_index=True,
        right_index=True,
        how="inner"
    ).shape[0]
    
    retained_true_link_percent = round((true_links_after_blocking/total_true_links) * 100, 2)
    
    print(f"{candidate_pairs.shape[0]:,} pairs after blocking: {search_space_reduction}% search space reduction.")
    print(f"{retained_true_link_percent}% true links retained after blocking.")
    
    return search_space_reduction, retained_true_link_percent 

We can evaluate the full blocker as such:

In [ ]:
%%time
_, _ = evaluate_blocking(full_blocker_pairs, df_A, df_B, df_ground_truth)

This makes sense. If we use a full blocker, we won't have reduced our search space at all. And, since we consider every possible candidate pair, this will include all true links.

However, let's see if we can do better. Let's experiment with a few sets of different blockers.

In [ ]:
indexer = rl.Index()

indexer.add(rl.index.Block("surname"))

candidate_pairs = indexer.index(df_A, df_B)

_, _ = evaluate_blocking(candidate_pairs, df_A, df_B, df_ground_truth)

In [ ]:
indexer = rl.Index()

indexer.add(rl.index.Block("surname"))
indexer.add(rl.index.Block("first_name"))

candidate_pairs = indexer.index(df_A, df_B)

_, _ = evaluate_blocking(candidate_pairs, df_A, df_B, df_ground_truth)

In [ ]:
indexer = rl.Index()

indexer.add(rl.index.Block("soundex_surname"))
indexer.add(rl.index.Block("soundex_firstname"))
indexer.add(rl.index.Block("nysiis_surname"))
indexer.add(rl.index.Block("nysiis_firstname"))

candidate_pairs = indexer.index(df_A, df_B)

_, _ = evaluate_blocking(candidate_pairs, df_A, df_B, df_ground_truth)

In [ ]:
indexer = rl.Index()

indexer.add(rl.index.Block("soundex_surname"))
indexer.add(rl.index.Block("soundex_firstname"))
indexer.add(rl.index.Block("nysiis_surname"))
indexer.add(rl.index.Block("nysiis_firstname"))
indexer.add(rl.index.Block("ssid_last3"))
indexer.add(rl.index.Block("date_of_birth"))

candidate_pairs = indexer.index(df_A, df_B)

_, _ = evaluate_blocking(candidate_pairs, df_A, df_B, df_ground_truth)

## Comparing

After we're reasonably satisifed with our blockers, we can move on to comparing our candidate record pairs. Recall that in the comparison step, for each candidate record pair, we compare their attributes to generate a comparison vector. Once again, we'll use [`recordlinkage`](https://github.com/J535D165/recordlinkage) to define our comparators. `recordlinkage` offers a variety of built-in comparators to use for string, numeric, and datetime fields.

* We can use exact comparators for our phonetic encoding fields.
* We'll use Jaro-Winkler comparison for the name fields, as this comparison approach is specifically designed for comparison of names.
* For the other string fields, we'll opt for Damerau-Levenshtein, which does a nice job in accomodating data entry typos.
* For the DOB, we'll use a date comparison.

In [ ]:
%%time

comparer = rl.Compare()

# Phonetic encodings.
comparer.add(rl.compare.Exact("soundex_surname", "soundex_surname", label="soundex_surname"))
comparer.add(rl.compare.Exact("soundex_firstname", "soundex_firstname", label="soundex_firstname"))
comparer.add(rl.compare.Exact("nysiis_surname", "nysiis_surname", label="nysiis_surname"))
comparer.add(rl.compare.Exact("nysiis_firstname", "nysiis_firstname", label="nysiis_firstname"))

# First & last name.
comparer.add(rl.compare.String("surname", "surname", method="jarowinkler", label="last_name"))
comparer.add(rl.compare.String("first_name", "first_name", method="jarowinkler", label="first_name"))

# Address.
comparer.add(rl.compare.String("address_1", "address_1", method="damerau_levenshtein", label="address_1"))
comparer.add(rl.compare.String("address_2", "address_2", method="damerau_levenshtein", label="address_2"))
comparer.add(rl.compare.String("suburb", "suburb", method="damerau_levenshtein", label="suburb"))
comparer.add(rl.compare.String("postcode", "postcode", method="damerau_levenshtein", label="postcode"))
comparer.add(rl.compare.String("state", "state", method="damerau_levenshtein", label="state"))

# Other fields.
comparer.add(rl.compare.Date("dob", "dob", label="date_of_birth"))
comparer.add(rl.compare.String("phone_number", "phone_number", method="damerau_levenshtein", label="phone_number"))
comparer.add(rl.compare.String("soc_sec_id", "soc_sec_id", method="damerau_levenshtein", label="ssn"))

features = comparer.compute(candidate_pairs, df_A, df_B)

You can see that the output of the compare step is a collection of comparison/feature vectors, one for each candidate record pair. `recordlinkage` returns these vectors as a pandas Dataframe, indexed on the record pair ids.

In [ ]:
features

Here's a look at an individual comparison vector:

In [ ]:
display(features.iloc[0].name)
display(features.iloc[0])

## Add labels to feature vectors

We've generated our comparison/feature vectors, now we're ready to classify! To begin, we'll add our ground truth labels to the features DataFrame. Note that `df_ground_truth` just contains the true links, so we'll use a left join and then `fillna` with `False` for any records that are not true links.

In [ ]:
df_labeled_features = pd.merge(
    features,
    df_ground_truth,
    on=["person_id_A", "person_id_B"],
    how="left"
)

df_labeled_features["ground_truth"].fillna(False, inplace=True)
df_labeled_features.head()

## Calculate SimSum Scores

Once again, SimSum is the simplest approach to linking classification. To generate our scores for the candidate record pairs, we simply sum the values each attribute comparison score into a single score for each record.

In [ ]:
df_labeled_features["simsum"] = df_labeled_features.drop("ground_truth", axis=1).sum(axis=1)
df_labeled_features.head()

## Choosing a SimSum Classification Threshold

Now that we've generated scores for all of our candidate record pairs, the next step is to determine a threshold at which we can classify a record pair as a link, or not-a-link. To do this, it's first helpful to look at the score distribution.

### "Model" Score Distribution

We can see a pretty clear boundary between not-links and links when it comes to the SimSum score. There's a bit of an overlap from 7 - 9.5, but it looks like we'll probably want to set the cutoff somewhere in that range.

In [ ]:
tutorial.plot_model_score_distribution(
    df_labeled_features,
    score_column_name="simsum",  
)

### Precision and Recall at Varying Thresholds

Next, we'll take a look at the calculated precision and recall at varying model score thresholds. Below is a function which calculates precision and recall for a range of scores.

In [ ]:
def evaluate_linking(
    df: pd.DataFrame,
    score_column_name: Optional[str] = "score",
    ground_truth_column_name: Optional[str] = "ground_truth",
) -> Tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame]:
    """ Use model results to calculate precision & recall metrics.
    
        Args:
            df: dataframe containing model scores, and ground truth labels
                indexed on df_left index, df_right index
            score_column_name: Optional string name of column containing model scores 
            ground_truth_column_name: Optional string name of column containing ground
                truth values
                
        Returns:
            Tuple containing:
                pandas dataframe with precision and recall evaluation data
                at varying score thresholds
    """
    eval_data = []
    max_score = max(1, max(df[score_column_name]))
    
    # Calculate eval data at threshold intervals from zero to max score. 
    # Max score is generally 1.0 if using a ML model, but with SimSum it
    # can get much larger.
    for threshold in np.linspace(0, max_score, 50):
        tp = df[(df[score_column_name] >= threshold) & (df[ground_truth_column_name] == True)].shape[0]
        fp = df[(df[score_column_name] >= threshold) & (df[ground_truth_column_name] == False)].shape[0]
        tn = df[(df[score_column_name] < threshold) & (df[ground_truth_column_name] == False)].shape[0]
        fn = df[(df[score_column_name] < threshold) & (df[ground_truth_column_name] == True)].shape[0]
        
        precision = tp / (tp + fp)
        recall = tp / (tp + fn)
        f1 = 2 * ((precision * recall)/(precision + recall))
        
        eval_data.append(
            {
                "threshold" : threshold,
                "tp" : tp,
                "fp" : fp,
                "tn" : tn,
                "fn" : fn,
                "precision" : precision,
                "recall" : recall,
                "f1" : f1
            }
        )
        
    return pd.DataFrame(eval_data)

In [ ]:
df_eval = evaluate_linking(
    df=df_labeled_features,
    score_column_name = "simsum",
)

In [ ]:
df_eval.head()

The plot precision and recall at varying score thresholds reinforces what we noted earlier in the score distribution - that our most suitable cutoff is in the range of 7 to 9.5. It relies on your own particular use case to determine exactly where the cutoff should be set (e.g. Is recall more important than precision, or vice versa?).

In [ ]:
tutorial.plot_precision_recall_vs_threshold(df_eval)

We can also take a look at the F1 score at varying model thresholds. F1 is the harmonic mean of precision and recall, which provides us with a single figure to consider.

In [ ]:
tutorial.plot_f1_score_vs_threshold(df_eval)

## Examining Individual Links

Another way to gain insight into the performance of link classification is examining individual links (including their original attribute values) in score ranges of interest. This can be particularly helpful where you see overlap of classes in your score distribution - i.e. where you see highly scored non-links and poorly scored true links. These cases can highlight model confusion, and shed light on potential feature improvements.

Below, we've:
* Defined a helper function to join scored pairs with their original entity attribute data
* Captured the top scoring non-links (negatives) as well as the lowest scoring true links (positives)

In [ ]:
def augment_scored_pairs(
    df: pd.DataFrame,
    df_left: pd.DataFrame,
    df_right: pd.DataFrame,
    score_column_name: Optional[str] = "score",
    ground_truth_column_name: Optional[str] = "ground_truth"
) -> pd.DataFrame:
    """ Augment scored pairs with original entity attribute data.
    
        Args:
            df: dataframe containing pairs for examination that includes
                model scores and ground truth labels, and is indexed on
                df_left index, df_right index
            df_left: dataframe containing attributes for "left"-linked entities
            df_right: dataframe containing attributes for "right"-linked entities
            score_column_name: Optional string name of column containing model scores 
            ground_truth_column_name: Optional string name of column containing ground
                truth values
                
        Returns:
            Tuple containing:
                pandas dataframe containing pairs augmented with original entity attributes 
    """
    
    df = df[[score_column_name, ground_truth_column_name]]

    # Suffix our original attribute fields for display convenience when
    # we examine the links in the notebook.
    df_left = df_left.copy()
    df_left.columns = df_left.columns.map(lambda x: str(x) + '_A')

    df_right = df_right.copy()
    df_right.columns = df_right.columns.map(lambda x: str(x) + '_B')
    
    # Join the original link entity data via the dataframe indices.
    # This gives us the model score as well as the actual human-readable attributes
    # for each link.
    df_augmented_pairs = pd.merge(
        df,
        df_left,
        left_on=df_left.index.name,
        right_index=True,
    )

    # Join data from right entities.
    df_augmented_pairs = pd.merge(
        df_augmented_pairs,
        df_right,
        left_on=df_right.index.name,
        right_index=True,
    ) 
    
    return df_augmented_pairs

In [ ]:
display_cols = [
    "first_name", "surname",
    "street_number", "address_1", "address_2", "suburb", "postcode", "state",
    "date_of_birth", "age", "phone_number", "soc_sec_id",
    "soundex_surname", "soundex_firstname",
    "nysiis_surname", "nysiis_firstname",
]

display_cols = [[f"{col}_A", f"{col}_B"] for col in display_cols]
display_cols = list(itertools.chain.from_iterable(display_cols))

### Top Scoring Non-Links

In [ ]:
df_top_scoring_negatives = df_labeled_features[
    df_labeled_features["ground_truth"] == False
][["simsum", "ground_truth"]].sort_values("simsum", ascending=False).head(n=10)

df_top_scoring_negatives = augment_scored_pairs(df_top_scoring_negatives, df_A, df_B, score_column_name="simsum")

with pd.option_context('display.max_columns', None):
    display(df_top_scoring_negatives[["simsum", "ground_truth"] + display_cols])

### Lowest Scoring True Links

In [ ]:
df_lowest_scoring_positives = df_labeled_features[
    df_labeled_features["ground_truth"] == True
][["simsum", "ground_truth"]].sort_values("simsum").head(n=10)

df_lowest_scoring_positives = augment_scored_pairs(df_lowest_scoring_positives, df_A, df_B, score_column_name="simsum")

with pd.option_context('display.max_columns', None):
    display(df_lowest_scoring_positives[["simsum", "ground_truth"] + display_cols])